## SAS Function to Python using GenAI

### Install packages

In [ ]:
%pip install scikit-learn
%pip install openai
# Install required modules and set required tokens
%pip install openai python-dotenv
%pip install requests

## Setup Env

In [1]:
# Load the .env file in a Jupyter notebook:
%load_ext dotenv
%dotenv

In [20]:
import pandas as pd
import openai
import os
import json
from pprint import pprint

### Setup proxy

In [3]:
import requests
from requests.auth import HTTPProxyAuth

/Users/muralikorrapati/pythonPractice/MLearning/llm_sas_to_python/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Configure your proxy
proxy = "http://95.164.128.235:9943"
proxy_auth = HTTPProxyAuth('M0k0EN', 'ZSNkKy')

# Configure a session to use the proxy
session = requests.Session()
session.proxies = {'http': proxy, 'https': proxy}
session.auth = proxy_auth

In [5]:
openai.api_key = os.environ.get('OPENAI_API_KEY')
# openai.organization = os.environ.get('OPENAI_ORG_ID')
if os.environ.get('use_proxy'):
    openai.session = session


In [46]:
# https://sebscholl.medium.com/using-chatgpt-in-jupyter-notebooks-c6144fef79d8
class ChatSession:
    """
    Class for running a chat session. 
    
    Create and use a new chat by running:
        chat = ChatSession('new-chat-name', 'system-role-message')

    Load an existing chat by running:
        chat = ChatSession('existing-chat-name')
    """
    model = 'gpt-3.5-turbo-16k-0613'
    
    def __init__(self, chat_name, system_role=None):
        self.name = chat_name
        self.path = f"./chats/{chat_name}.json"

        if system_role is not None:
            with open(self.path, "w") as outfile:
                outfile.write(json.dumps([system_role], indent=4))
            print(f"Chat {self.path} created 💬")
        else:
            print(f"Active chat set to {self.name} 💬")
            
    def chat(self, message):
        # Retrieve chat history
        with open(self.path) as openchat:
            chat_history = json.load(openchat)
        # Update chat history before completion
        chat_history.append({'role': 'user', 'content': message})
        # Run completeion
        chat_completion = openai.chat.completions.create(model=self.model, messages=chat_history)
        
        # Save message to chat
        chat_history.append({'role': 'system', 'content':chat_completion.choices[0].message.content})
        # Handle length finish reason
        if chat_completion.choices[0].finish_reason == 'length':
            chat_completion = openai.chat.completions.create(model=self.model, messages=chat_history)
            
        # Save the chat history
        with open(self.path, "w") as outfile:
            json.dump(chat_history, outfile)
        # Print new dialog.
        for msg in chat_history[-2:]:
            print(f"ROLE: {msg['role']}")
            print("------------------------")
            print(f"{msg['content']}")
            # for line in msg["content"]:
            #     print(f"\t{line}")

### Start chat session

In [47]:
c1 = ChatSession("philosophical-chat", { "role": "system", "content": "You are an AI assistant that converts SAS code to Python." }) 

Chat ./chats/philosophical-chat.json created 💬


In [48]:
c1.chat("""
/* Assign some macro variables */
%let var1 = test;
%let id = 39;

/* Print to log */
%put var1 = &var1.;
%put id = &id.;
/* Use the macro variables in a data step to create table */
data test;
       m&id. = 333;
       v = "&var1.";
run;
""")

ROLE: user

/* Assign some macro variables */
%let var1 = test;
%let id = 39;

/* Print to log */
%put var1 = &var1.;
%put id = &id.;
/* Use the macro variables in a data step to create table */
data test;
       m&id. = 333;
       v = "&var1.";
run;

ROLE: system
# Assign some macro variables
var1 = "test"
id = 39

# Print to log
print("var1 =", var1)
print("id =", id)

# Use the macro variables in a data step to create table
test = pd.DataFrame({
    "m" + str(id): [333],
    "v": [var1]
})
